<a href="https://colab.research.google.com/github/Kacper-W-Kozdon/notebook-testing-ivy/blob/main/Sarcasm_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [3]:
from google.colab import files
from google.colab import userdata
import os

In [11]:
files.upload();

In [5]:
!mkdir ~/.kaggle

In [6]:
!cp kaggle.json ~/.kaggle/

In [8]:
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
!kaggle datasets list

ref                                                          title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
syedanwarafridi/vehicle-sales-data                           Vehicle Sales Data                                  19MB  2024-02-21 20:16:17          10645        196  1.0              
arnavvvvv/spotify-music                                      Top Spotify Songs                                   47KB  2024-03-06 05:20:29           2156         42  1.0              
yaminh/smartphone-sale-dataset                               Smartphones Sales Dataset                           65KB  2024-03-03 16:48:27           2459         35  0.88235295       
tarunrm09/climate-change-indicators                          Climate change Indi

In [10]:
!kaggle datasets download -d danofer/sarcasm

 98% 212M/216M [00:06<00:00, 44.9MB/s]
100% 216M/216M [00:06<00:00, 35.5MB/s]


In [ ]:
!git clone https://github.com/Kacper-W-Kozdon/demos.git
!git clone https://github.com/Kacper-W-Kozdon/ivy.git

Cloning into 'demos'...
remote: Enumerating objects: 3152, done.
remote: Counting objects: 100% (608/608), done.
remote: Compressing objects: 100% (229/229), done.
remote: Total 3152 (delta 440), reused 421 (delta 348), pack-reused 2544
Receiving objects: 100% (3152/3152), 16.26 MiB | 23.29 MiB/s, done.
Resolving deltas: 100% (1813/1813), done.
fatal: destination path 'ivy' already exists and is not an empty directory.


In [ ]:
!pip install -U -q ivy mlflow datasets>=2.14.5 nlp 2>/dev/null
import ivy

In [ ]:
# Import necessary libraries
import pandas as pd  # For data manipulation and analysis
import gc  # For garbage collection to manage memory
import re  # For regular expressions
import numpy as np  # For numerical operations and arrays

import warnings  # For handling warnings
warnings.filterwarnings("ignore")  # Ignore warning messages

import torch  # PyTorch library for deep learning
from transformers import AutoModel, AutoTokenizer  # Transformers library for natural language processing
from transformers import TextDataset, LineByLineTextDataset, DataCollatorForLanguageModeling, \
pipeline, Trainer, TrainingArguments, DataCollatorWithPadding  # Transformers components for text processing
from transformers import AutoModelForSequenceClassification  # Transformer model for sequence classification

from nlp import Dataset  # Import custom 'Dataset' class for natural language processing tasks
from imblearn.over_sampling import RandomOverSampler  # For oversampling to handle class imbalance
import datasets  # Import datasets library
from datasets import Dataset, Image, ClassLabel  # Import custom 'Dataset', 'ClassLabel', and 'Image' classes
from transformers import pipeline  # Transformers library for pipelines
from bs4 import BeautifulSoup  # For parsing HTML content

import matplotlib.pyplot as plt  # For data visualization
import itertools  # For working with iterators
from sklearn.metrics import (  # Import various metrics from scikit-learn
    accuracy_score,  # For calculating accuracy
    roc_auc_score,  # For ROC AUC score
    confusion_matrix,  # For confusion matrix
    classification_report,  # For classification report
    f1_score  # For F1 score
)

from datasets import load_metric  # Import load_metric function to load evaluation metrics

from tqdm import tqdm  # For displaying progress bars
tqdm.pandas()  # Enable progress bars for pandas operations

In [ ]:
df = pd.read_csv("/content/demos/Contributor_demos/Sarcasm Detection/train-balanced-sarcasm.csv")


In [ ]:

df = df.drop_duplicates()
df = df[['comment', 'label']]
df = df[~df['comment'].isnull()]
df = df[~df['label'].isnull()]
print(df.sample(15))

                                                  comment  label
531718  because the american political model can be us...      1
302525  Mary Kate and Ashely: Island Dance Party was c...      0
698658  Egalitarianism is equality in all respects, no...      0
411629  So you'd rather wait until a news article with...      0
978426  Oh dear god how DARE they have a contest where...      1
622068    Have you considered filming it and showing him?      0
346420  One is younger and has made it to the NHL, and...      1
640505  Poor guy in the back trying to see around the ...      0
129036  Why is Booker better at defense than Justise W...      0
841552             you've never joined the thieves guild?      0
676458  **"Backup Exec is not shit"** ^^^^The ^^^^worl...      0
870757  Why don't we all just buy the new iPhone so th...      1
491654                               I know that feel :,(      0
531004  Too late, I've already heard of the accident e...      1
232621  The irony of refe

In [ ]:
dataset = Dataset.from_pandas(df)
del df
gc.collect()


151

In [ ]:
def get_training_corpus():
  training_corpus = (
    dataset["comment"][i : i + 1000]
    for i in range(0, len(dataset["comment"]), 1000)
  )
  return training_corpus

training_corpus = get_training_corpus()


In [ ]:
old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [ ]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

In [ ]:
tokenizer.save_pretrained("Sarcasm_Detection-Tokenizer")

('Sarcasm_Detection-Tokenizer/tokenizer_config.json',
 'Sarcasm_Detection-Tokenizer/special_tokens_map.json',
 'Sarcasm_Detection-Tokenizer/vocab.json',
 'Sarcasm_Detection-Tokenizer/merges.txt',
 'Sarcasm_Detection-Tokenizer/added_tokens.json',
 'Sarcasm_Detection-Tokenizer/tokenizer.json')

In [ ]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [ ]:

print(dataset["comment"][1], dataset["label"][1])
tokens = tokenizer(dataset["comment"][1:5], padding=True)
print(tokens)
decoded_string = tokenizer.decode(tokens.get("input_ids")[0])
print(decoded_string)

You do know west teams play against west teams more than east teams right? 0
{'input_ids': [[446, 363, 465, 5140, 2347, 528, 1095, 5140, 2347, 506, 525, 5338, 2347, 539, 31, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000], [741, 631, 42310, 4887, 1953, 12, 392, 1014, 18461, 312, 10837, 371, 15283, 12, 265, 9009, 1698, 515, 5277, 284, 14656, 742, 17], [677, 3195, 781, 320, 1460, 3407, 302, 265, 470, 9154, 25745, 12973, 2, 1671, 354, 14, 52000, 52000, 52000, 52000, 52000, 52000, 52000], [41, 609, 713, 479, 302, 646, 7910, 14, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}
You do know west teams play against west teams more

In [ ]:
from ivy.stateful.module import Module
from ivy.stateful.sequential import Sequential
from ivy.stateful.layers import *
from ivy.stateful.losses import *
from ivy.stateful.optimizers import *
from ivy.stateful.activations import *
from ivy.stateful.initializers import *
from ivy.stateful.norms import *